In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

class ConvM(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2        

        norm_layer = nn.BatchNorm2d
        super(ConvM, self).__init__(            

            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            norm_layer(out_planes),
            nn.ReLU(inplace=True),
        )

        conv=nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False)

        replace_weight = torch.tensor([[[1,0],
                                       [0,1]],
                                       [[0,1],
                                       [1,0]]])
        #replace_bias = torch.tensor([100000])

        conv.weight.data=replace_weight
        #conv.bias.data=replace_bias

        print('weight tensor of conv layer:\n{}'.format(conv.weight))
        #print('bias tensor of conv layer:\n{}'.format(conv.bias)) 



class ConvNet(nn.Module):
    def __init__(self, n_class):
        super(ConvNet, self).__init__()

        self.conv = nn.Sequential(
            ConvM(2, 2, 2, 1)                                         
        )

        self.fc = nn.Linear(2, n_class)

    def forward(self, x):
        x = self.conv(x)
        print("conv_x:",x)                       
        print("conv_x_size:",x.size())                         #1*2*2*2
        #print("conv_x.shape[0]:",x.shape[0])
        #print("conv_x.shape[1]:",x.shape[1])
        #print("conv_x.shape[2]:",x.shape[2])
        #print("conv_x.shape[3]:",x.shape[3])

        x = nn.functional.adaptive_avg_pool2d(x, 1)
        print("adaptive_avg_pool2d_x:",x)                                    
        print("adaptive_avg_pool2d_x_size:",x.size())          #1*2*1*1
        #print("adaptive_avg_pool2d_x.shape[0]:",x.shape[0])
        #print("adaptive_avg_pool2d_x.shape[1]:",x.shape[1])
        #print("adaptive_avg_pool2d_x.shape[2]:",x.shape[2])
        #print("adaptive_avg_pool2d_x.shape[3]:",x.shape[3])

        x = x.reshape(x.shape[0], -1) 
        print("reshape_x:",x)
        print("reshape_x_size:",x.size())       #轉成二維 1*2(降維)                
        #print("reshape_x:",x.shape[0])
        #print("reshape_x:",x.shape[1])

        output = self.fc(x)           
        print("output:",output)               
        print("output_size:",output.size())     #轉成二維 1*4(分類)

        return output

dummy_inpurt = torch.rand(1,2,3,3)
print("dummy_inpurt:",dummy_inpurt)

model = ConvNet(n_class=4)
out = model(dummy_inpurt) 

dummy_inpurt: tensor([[[[0.0568, 0.3616, 0.8370],
          [0.5148, 0.6241, 0.9322],
          [0.9141, 0.0634, 0.3445]],

         [[0.7159, 0.5276, 0.8828],
          [0.0875, 0.0706, 0.6873],
          [0.3516, 0.8273, 0.1722]]]])
weight tensor of conv layer:
Parameter containing:
tensor([[[1, 0],
         [0, 1]],

        [[0, 1],
         [1, 0]]], requires_grad=True)
conv_x: tensor([[[[0.8103, 0.0000],
          [0.0000, 1.0176]],

         [[0.2088, 1.0368],
          [0.0000, 0.4031]]]], grad_fn=<ReluBackward1>)
conv_x_size: torch.Size([1, 2, 2, 2])
adaptive_avg_pool2d_x: tensor([[[[0.4570]],

         [[0.4122]]]], grad_fn=<MeanBackward1>)
adaptive_avg_pool2d_x_size: torch.Size([1, 2, 1, 1])
reshape_x: tensor([[0.4570, 0.4122]], grad_fn=<ViewBackward>)
reshape_x_size: torch.Size([1, 2])
output: tensor([[-0.4866,  0.2273,  0.1762,  0.1512]], grad_fn=<AddmmBackward>)
output_size: torch.Size([1, 4])


In [2]:
import torch  
import torch.nn as nn  
import torch.nn.functional as F 

class ConvM(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2        

        norm_layer = nn.BatchNorm2d
        super(ConvM, self).__init__(            

            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            norm_layer(out_planes),
            nn.ReLU(inplace=True),
        )

class ConvNet(nn.Module):
    def __init__(self, n_class=3):
        super(ConvNet, self).__init__()

        self.conv = nn.Sequential(
            ConvM(3, 32, 5, 2),        #32*5*5            
            ConvM(32, 64, 5, 2),       #64*3*3
            ConvM(64, 128, 3, 1),      #128*3*3
            ConvM(128, 64, 3, 1),      #64*3*3
            ConvM(64, 32, 3, 1),       #32*3*3     
        )        
        self.fc = nn.Linear(32, n_class)
    def forward(self, x):
        x = self.conv(x)
        #print(x)                       #32*3*3  

        x = nn.functional.adaptive_avg_pool2d(x, 1)
        #print(x)                       #1*32*1*1             
        print(x.size())
        print(x.shape[0])
        print(x.shape[1])
        print(x.shape[2])
        print(x.shape[3])

        x = x.reshape(x.shape[0], -1)  #轉成二維 1*32
        #print(x)
        print(x.size())
        print(x.shape[0])
        print(x.shape[1])

        output = self.fc(x)            #轉成二維 1*3
        print(output)

        return output

model = ConvNet(n_class=3)
dummy_inpurt = torch.rand(1,3,10,10)  
out = model(dummy_inpurt) 

torch.Size([1, 32, 1, 1])
1
32
1
1
torch.Size([1, 32])
1
32
tensor([[0.3220, 0.3607, 0.2978]], grad_fn=<AddmmBackward>)
